Source: https://github.com/BlackPianoCat/EasyOpenMM/tree/main

# Imports

In [7]:
import copy
import numpy as np
import time
from utils import *
import openmm as mm
import openmm.unit as u
from tqdm import tqdm
from sys import stdout
from mdtraj.reporters import HDF5Reporter
from openmm.app import PDBFile, PDBxFile, ForceField, Simulation, PDBReporter, PDBxReporter, DCDReporter, StateDataReporter, CharmmPsfFile,  DCDFile
import random
import pyvista as pv
import mdtraj as md
from hilbert import decode, encode

# Initial structure generation functions

In [12]:
def line(n):
    points = []
    for i in range(n):
        points.append([i, 0, 0])
    return np.array(points)

def random_walk(n):
    points=[[0,0,0] for _ in range(n) ]
    for i in range(1,n):
        val_x = random.randint(-1,1)
        val_y = random.randint(-1,1)
        val_z = random.randint(-1,1)
        base = points[i-1]
        points[i] = [base[0]+val_x,base[1]+val_y,base[2]+val_z]
    return points

def random_walk2(n):
    steps_x = np.random.normal(loc=0, scale=1, size=n)
    steps_y = np.random.normal(loc=0, scale=1, size=n)
    steps_z = np.random.normal(loc=0, scale=1, size=n)
    positions_x = np.cumsum(steps_x)
    positions_y = np.cumsum(steps_y)
    positions_z = np.cumsum(steps_z)
    return np.column_stack((positions_x,positions_y,positions_z))


def hilbert_curve2d(n):
    points=[[0,0,0]] # starting point

    instructions_dict = {"A": ["D","A","A","B"],
                         "B": ["C","B","B","A"],
                         "C": ["B","C","C","D"],
                         "D": ["A","D","D","C"]} # production rules from Wikipedia
    
    n_iter = np.ceil(n/4) #number of blocks needed to connect given number of points (n)
    stack = ["A"]  # list of blocks
    
    # defining blocks
    while len(stack) < n_iter:
        instructions_list=[]
        while len(stack) > 0:
            instructions_list = np.concatenate((instructions_list,np.array(instructions_dict[stack.pop()])))
        stack = instructions_list[::-1].tolist()

    i = 1
    while i <= n_iter:
        current_stage = stack.pop() #get current block (stage)
        base = points[-1] #get the base point, we will start drawing from start's coordinates

        if current_stage == 'A': #draw A
            points.append([base[0], base[1]+1,base[2]])
            points.append([base[0]+1, base[1]+1 ,base[2]])
            points.append([base[0]+1, base[1],base[2]])
            base = points[-1]
            next_stage = stack[-1] if len(stack) >0 else None
            if next_stage  == "A" or next_stage == "D": # prepare base for next stage
                points.append([base[0]+1, base[1],base[2]])
            else:
                points.append([base[0], base[1]-1,base[2]])

        if current_stage == 'B':
            points.append([base[0]-1, base[1],base[2]])
            points.append([base[0]-1, base[1]-1,base[2]])
            points.append([base[0], base[1]-1,base[2]])
            base = points[-1]
            next_stage = stack[-1] if len(stack) >0 else None
            if next_stage  == "B" or next_stage == "C":
                points.append([base[0], base[1]-1,base[2]])
            else:
                points.append([base[0]+1, base[1],base[2]])

        if current_stage == 'C':
            points.append([base[0], base[1]-1,base[2]])
            points.append([base[0]-1, base[1]-1,base[2]])
            points.append([base[0]-1, base[1],base[2]])
            base = points[-1]
            next_stage = stack[-1] if len(stack) >0 else None
            if next_stage  == "C" or next_stage == "B":
                points.append([base[0]-1, base[1],base[2]])
            else:
                points.append([base[0], base[1]+1,base[2]])

        if current_stage == 'D':
            points.append([base[0]+1, base[1],base[2]])
            points.append([base[0]+1, base[1]+1,base[2]])
            points.append([base[0], base[1]+1,base[2]])
            base = points[-1]
            next_stage = stack[-1] if len(stack) >0 else None
            if next_stage  == "D" or next_stage == "A":
                points.append([base[0], base[1]+1,base[2]])
            else:
                points.append([base[0]-1, base[1],base[2]])
        
        i +=1
        
    while len(points) > n:
        points.pop() # delete redundant points
    return points

def hilbert_curve3d(n):
    num_dims = 3

    num_bits = int(np.ceil(np.log2(n)/num_dims)) # number of bits per dimention
    
    max_h = 2**(num_bits*num_dims) # this is the maximum number of nodes in our cube. It wont allways be full 
    
    hilberts = np.arange(n)
    locs = decode(hilberts, num_dims, num_bits)

    return locs    


def helisa(n, radius = 1, step_z = 0.1, n_rotations = 5):
    steps_z = np.repeat(step_z,n)
    angles = np.linspace(0,n_rotations* 2*np.pi, n, endpoint= False)
    x = radius * np.cos(angles)
    y = radius * np.sin(angles)
    z = np.cumsum(steps_z)

    return np.column_stack((x,y,z))

def sphere(n, radius = 1):
    angles_xz = np.random.uniform(0,np.pi, n)
    angles_xy = np.random.uniform(0,2*np.pi, n)
    x = radius * np.cos(angles_xy) * np.cos(angles_xz)
    y = radius * np.cos(angles_xy) * np.sin(angles_xz)
    z = radius * np.sin(angles_xy)

    return np.column_stack((x,y,z))

def cube(n, edge_len = 1):
    x = np.random.uniform(0,edge_len,n)
    y = np.random.uniform(0,edge_len,n)
    z = np.random.uniform(0,edge_len,n)

    return np.column_stack((x,y,z))

def cube_outer(n, edge_len = 1):
    walls = np.random.randint(1,7,n)
    points = []
    for wall in walls:
        x = 0 if wall == 1 else edge_len if wall == 3 else np.random.uniform(0,edge_len)
        y = 0 if wall == 2 else edge_len if wall == 4 else np.random.uniform(0,edge_len)
        z = 0 if wall == 5 else edge_len if wall == 6 else np.random.uniform(0,edge_len)
        points.append([x,y,z])
    return points

# Simulation

In [13]:
# 0. Generate some initial structure
N_beads=100
points = helisa(N_beads)
write_mmcif(points,'init_struct.cif')
generate_psf(N_beads,'LE_init_struct.psf')

# 1. Define System
pdb = PDBxFile('init_struct.cif')
forcefield = ForceField('forcefields/classic_sm_ff.xml')
system = forcefield.createSystem(pdb.topology, nonbondedCutoff=1*u.nanometer)
integrator = mm.LangevinIntegrator(310, 0.05, 100 * mm.unit.femtosecond)

# 2. Define the forcefield
# 2.1. Harmonic bond borce between succesive beads
bond_force = mm.HarmonicBondForce()
system.addForce(bond_force)
for i in range(system.getNumParticles() - 1):
    bond_force.addBond(i, i + 1, 0.1, 300000.0)
bond_force.addBond(10, 70, 0.001, 0.001) # connect bead 10 with bead 70

#2.2. Harmonic angle force between successive beads so as to make chromatin rigid
angle_force = mm.HarmonicAngleForce()
system.addForce(angle_force)
for i in range(system.getNumParticles() - 2):
    angle_force.addAngle(i, i + 1, i + 2, np.pi, 0.001)
    
# 3. Minimize energy
simulation = Simulation(pdb.topology, system, integrator)
simulation.reporters.append(StateDataReporter(stdout, 10, step=True, totalEnergy=True, potentialEnergy=True, temperature=True))
simulation.reporters.append(DCDReporter('stochastic_LE.dcd', 10))
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy(tolerance=0.001)
state = simulation.context.getState(getPositions=True)
PDBxFile.writeFile(pdb.topology, state.getPositions(), open('minimized.cif', 'w')) # save minimized file

# 4. Run md simulation
simulation.context.setVelocitiesToTemperature(310, 0)
simulation.step(10000)
state = simulation.context.getState(getPositions=True)
PDBxFile.writeFile(pdb.topology, state.getPositions(), open('after_sim.cif', 'w')) # save minimized file
PDBFile.writeFile(pdb.topology, state.getPositions(), open('after_sim.pdb', 'w')) # save minimized file
df = DCDFile(open("after_sim.dcd", "wb"),pdb.topology, dt = 100 * mm.unit.femtosecond)
df.writeModel(state.getPositions(), pdb.topology.getUnitCellDimensions(), pdb.topology.getPeriodicBoxVectors())

#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,107.27322387695312,316.13366567716,169.15917313518588
20,82.96114349365234,320.29261162132025,192.21828011759123
30,89.17096710205078,337.1468686237931,200.8393647802227
40,94.56158447265625,336.2572063356638,195.75287303016904
50,91.58721923828125,357.9594160877168,215.7387975285268
60,82.23358154296875,343.2710475176573,211.4181197788637
70,81.62876892089844,351.0761366188526,218.22942406125458
80,96.01492309570312,356.9387115314603,211.32605065211126
90,87.42857360839844,379.08236931450665,236.2147398432575
100,103.54850006103516,391.5979905053973,233.29555949226028
110,87.23733520507812,406.9284926354885,258.9226154241774
120,89.27845764160156,412.5447427332401,261.81816440326367
130,103.89022827148438,423.42676820605993,258.7973884184208
140,104.93695831298828,419.017266407609,254.37830523289568
150,127.93624877929688,434.52535669505596,248.31107097290794
160,106.31491088867188,454.4865112155676,281

# Visualization

In [14]:
traj = md.load("after_sim.cif")

positions = traj.xyz

mesh = pv.PolyData(positions[0])

# Create PyVista plotter
plotter = pv.Plotter(notebook=True)

# Add mesh to the plotter
plotter.add_mesh(mesh, color="blue", point_size=5)

# Create lines between consecutive points
lines = pv.lines_from_points(positions[0])

# Add lines to the plotter
plotter.add_mesh(lines, color="red", line_width=2)

# Show the plotter using Trame's notebook backend
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:52478/index.html?ui=P_0x1f9595e7a00_1&reconnect=auto" class="pyvis…

### Testing initial structure generation function

In [15]:
positions = cube_outer(1000)

mesh = pv.PolyData(positions)

# Create PyVista plotter
plotter = pv.Plotter(notebook=True)

# Add mesh to the plotter
plotter.add_mesh(mesh, color="blue", point_size=5)

# Create lines between consecutive points
lines = pv.lines_from_points(positions)

# Add lines to the plotter
plotter.add_mesh(lines, color="red", line_width=2)

# Show the plotter using Trame's notebook backend
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:52478/index.html?ui=P_0x1f95806f820_2&reconnect=auto" class="pyvis…

In [25]:
positions = hilbert_curve3d(4000)

mesh = pv.PolyData(positions)

# Create PyVista plotter
plotter = pv.Plotter(notebook=True)

# Add mesh to the plotter
plotter.add_mesh(mesh, color="blue", point_size=5)

# Create lines between consecutive points
lines = pv.lines_from_points(positions)

# Add lines to the plotter
plotter.add_mesh(lines, color="red", line_width=2)

# Show the plotter using Trame's notebook backend
plotter.show(jupyter_backend='trame')

c:\Users\Micha\anaconda3\envs\wb\lib\site-packages\pyvista\core\utilities\points.py:52: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value='<iframe src="http://localhost:52478/index.html?ui=P_0x1f947e019d0_8&reconnect=auto" class="pyvis…

# Testing multiple initial structures

In [27]:
initial_structure_gen_list = [line,random_walk,random_walk2,hilbert_curve2d, hilbert_curve3d, helisa, sphere, cube, cube_outer]

In [28]:
import os
def make_directory_if_not_exists(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

for f in initial_structure_gen_list:
    make_directory_if_not_exists(f'initial_structures_tests/{f.__name__}')

In [29]:
def run_simulation_on_initial_structure(initail_structure_gen_fun, N_beads):    
    points = initail_structure_gen_fun(N_beads)
    write_mmcif(points,f'initial_structures_tests/{initail_structure_gen_fun.__name__}/init_struct.cif')
    generate_psf(N_beads,f'initial_structures_tests/{initail_structure_gen_fun.__name__}/LE_init_struct.psf')

    # 1. Define System
    pdb = PDBxFile(f'initial_structures_tests/{initail_structure_gen_fun.__name__}/init_struct.cif')
    forcefield = ForceField('forcefields/classic_sm_ff.xml')
    system = forcefield.createSystem(pdb.topology, nonbondedCutoff=1*u.nanometer)
    integrator = mm.LangevinIntegrator(310, 0.05, 100 * mm.unit.femtosecond)

    # 2. Define the forcefield
    # 2.1. Harmonic bond borce between succesive beads
    bond_force = mm.HarmonicBondForce()
    system.addForce(bond_force)
    for i in range(system.getNumParticles() - 1):
        bond_force.addBond(i, i + 1, 0.1, 300000.0)
    bond_force.addBond(10, 70, 0.0001, 100000) # connect bead 10 with bead 70

    #2.2. Harmonic angle force between successive beads so as to make chromatin rigid
    angle_force = mm.HarmonicAngleForce()
    system.addForce(angle_force)
    for i in range(system.getNumParticles() - 2):
        angle_force.addAngle(i, i + 1, i + 2, np.pi, 10)

    # 3. Minimize energy
    simulation = Simulation(pdb.topology, system, integrator)
    simulation.reporters.append(StateDataReporter(stdout, 10, step=True, totalEnergy=True, potentialEnergy=True, temperature=True))
    simulation.reporters.append(DCDReporter(f'initial_structures_tests/{initail_structure_gen_fun.__name__}/stochastic_LE.dcd', 10))
    simulation.context.setPositions(pdb.positions)
    simulation.minimizeEnergy(tolerance=0.001)
    state = simulation.context.getState(getPositions=True)
    PDBxFile.writeFile(pdb.topology, state.getPositions(), open(f'initial_structures_tests/{initail_structure_gen_fun.__name__}/minimized.cif', 'w')) # save minimized file

    # 4. Run md simulation
    simulation.context.setVelocitiesToTemperature(310, 0)
    simulation.step(10000)
    state = simulation.context.getState(getPositions=True)
    PDBxFile.writeFile(pdb.topology, state.getPositions(), open(f'initial_structures_tests/{initail_structure_gen_fun.__name__}/after_sim.cif', 'w')) # save minimized file
    PDBFile.writeFile(pdb.topology, state.getPositions(), open(f'initial_structures_tests/{initail_structure_gen_fun.__name__}/after_sim.pdb', 'w')) # save minimized file
    df = DCDFile(open(f'initial_structures_tests/{initail_structure_gen_fun.__name__}/after_sim.dcd', "wb"),pdb.topology, dt = 100 * mm.unit.femtosecond)
    df.writeModel(state.getPositions(), pdb.topology.getUnitCellDimensions(), pdb.topology.getPeriodicBoxVectors())

In [30]:
for f in initial_structure_gen_list:
    run_simulation_on_initial_structure(f,100)

#"Step","Potential Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)"
10,3766.58203125,6657.067942008376,2341.047459291103
20,2986.454833984375,6457.011447865516,2810.855335088173
30,2188.3681640625,6153.585434094071,3211.4883456079874
40,2332.49658203125,5867.974203368649,2863.435823024271
50,2075.098876953125,5473.857007294893,2752.7046771502824
60,1728.7713623046875,5130.060434758663,2754.754524748343
70,1889.30078125,4947.284726262093,2476.706604373404
80,1727.5396728515625,4691.79051386565,2400.790771623876
90,1527.1634521484375,4381.71946400404,2311.9473006548496
100,1467.29638671875,4182.736078381538,2199.274927224091
110,1387.1065673828125,3956.5428909361362,2081.0246314214282
120,1698.9951171875,3789.071311801672,1692.7837450840775
130,1484.155517578125,3616.0181455016136,1726.6271978985374
140,1167.5693359375,3415.2802260294557,1820.454425633276
150,1369.7640380859375,3270.3557372540236,1539.3174386101148
160,1216.505615234375,3151.7874077558517,1567.4134603223324
1

In [31]:
plotter = pv.Plotter(shape=(len(initial_structure_gen_list), 2), notebook=True, window_size=(1000,2000))
for i,f in enumerate(initial_structure_gen_list):
    traj = md.load_dcd(f'initial_structures_tests/{f.__name__}/after_sim.dcd', top=f'initial_structures_tests/{f.__name__}/after_sim.pdb')
    positions = traj.xyz
    mesh = pv.PolyData(positions[0])
    plotter.subplot(i,1)
    plotter.add_text(f"{f.__name__}, after simulation", font_size=10) 
    plotter.add_mesh(mesh, color="blue", point_size=5)
    lines = pv.lines_from_points(positions[0])
    plotter.add_mesh(lines, color="red", line_width=2)

    traj = md.load(f'initial_structures_tests/{f.__name__}/init_struct.cif')
    positions = traj.xyz
    mesh = pv.PolyData(positions[0])
    plotter.subplot(i,0)
    plotter.add_text(f"{f.__name__}, initial structure", font_size=10) 
    plotter.add_mesh(mesh, color="blue", point_size=5)
    lines = pv.lines_from_points(positions[0])
    plotter.add_mesh(lines, color="red", line_width=2)
plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:52478/index.html?ui=P_0x1f9584494f0_9&reconnect=auto" class="pyvis…